In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-241840
Azure region: eastus2
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-241840


In [12]:
!pip install azureml-sdk[automl,notebooks,explain,train]

     |████████████████████████████████| 14.1 MB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 61.0 MB/s eta 0:00:01
     |████████████████████████████████| 420 kB 29.9 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 947 kB/s  eta 0:00:01
     |████████████████████████████████| 5.7 MB 51.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 50.1 MB/s eta 0:00:01
     |████████████████████████████████| 14.6 MB 63.9 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 60.8 MB/s eta 0:00:01
     |████████████████████████████████| 31.0 MB 57.4 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 322 kB 61.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 52.2 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 4.3 MB/s  eta 0:00:01
     |████████████████████████████████| 5.7 MB 58.2 MB/s eta 0:00:01
     |████████████████████████████

In [9]:
!pip install --upgrade azureml-sdk

     |████████████████████████████████| 137 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 8.6 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 248 kB 61.2 MB/s eta 0:00:01
     |████████████████████████████████| 38.2 MB 45.5 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 7.7 MB/s  eta 0:00:01
     |████████████████████████████████| 313 kB 66.5 MB/s eta 0:00:01
     |████████████████████████████████| 23.5 MB 39.1 MB/s eta 0:00:01
ERROR: azureml-widgets 1.51.0 has requirement azureml-core~=1.51.0, but you'll have azureml-core 1.53.0 which is incompatible.
ERROR: azureml-widgets 1.51.0 has requirement azureml-telemetry~=1.51.0, but you'll have azureml-telemetry 1.53.0 which is incompatible.
ERROR: azureml-training-tabular 1.51.0.post1 has requirement azureml-automl-core~=1.51.0, but you'll have azureml-automl-core 1.53.0 which is incompatible.
ERROR: azureml-train 1.51.0 

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "AzureProject"

# TODO: Create compute cluster
# Use vm_size = " Standard_D2_V2"in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# verify that cluser does not exist already
try:
    cpu_cluster=ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster= ComputeTarget.create(ws,cpu_cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# print detailed status for the current cluster
print(cpu_cluster.get_status().serialize())
    

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2023-09-26T13:21:18.608448+00:00', 'createdBy': {'userObjectId': '423a3759-db02-42f8-b72a-2f3312cb70da', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2023-09-26T13:24:28.590817+00:00', 'state': 'Running', 'vmSize': 'STANDARD_D2S_V3'}


In [15]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{"--C":choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
'--max_iter':choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                     script='train.py',
                     compute_target=cpu_cluster,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='validation_acc',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=16,
                                    max_concurrent_runs=4)

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run= exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [20]:
import joblib
# Get your best run and save the model from that run.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0,reverse =False, discard_no_metric=False))
### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metric:", best_run.get_metrics())
print("Run details",best_run.get_details() )
print("Best run file names", best_run.get_file_names())

[{'run_id': 'HD_41e9a83c-2a66-45ca-be5f-dbeeebf6f365_1', 'hyperparameters': '{"--C": 1000, "--max_iter": 200}', 'best_primary_metric': None, 'status': 'Preparing'}, {'run_id': 'HD_41e9a83c-2a66-45ca-be5f-dbeeebf6f365_2', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'best_primary_metric': None, 'status': 'Preparing'}, {'run_id': 'HD_41e9a83c-2a66-45ca-be5f-dbeeebf6f365_0', 'hyperparameters': '{"--C": 200, "--max_iter": 100}', 'best_primary_metric': None, 'status': 'Preparing'}, {'run_id': 'HD_41e9a83c-2a66-45ca-be5f-dbeeebf6f365_3', 'hyperparameters': '{"--C": 1, "--max_iter": 100}', 'best_primary_metric': None, 'status': 'Preparing'}]


AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###